# Number theory neat examples

<span style="font-size: 16pt; font-style: italic; font-weight: bold">Set 4 : </span>
<span style="font-size: 16pt; font-style: italic;">prime-pi, divisors, totient function</span>

Anton Antonov    
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
February, July 2025     

In [ ]:
#%js
    my $m = (7 .. 23).pick;
    my $s = 40;
    my $batch = (($s-1) * 2 * ($s-1) / 4).round;
    my @a = ((2..$s) X (2 .. 2 * $s)).hyper(:4degree, :$batch).map({ (prime-pi($_.tail) - prime-pi($_.head)) mod $m }).rotor(2 * $s-1);
    js-d3-matrix-plot(@a, width => 600, height => 300, :!grid-lines, color-palette => 'Plasma', :!tooltip)

----

## Introduction

**What is a neat example?** : Concise or straightforward code that produces compelling visual or textual outputs.

**Maybe:** We know *neat* when we see it?

The neat examples:

- Showcase Raku programming.
- Use functionalities of different Raku modules.
- Give interesting perspectives on what is computationally possible.

Showcased:
- All number theory functions are provided by ["Math::NumberTheory"](https://raku.land/zef:antononcube/Math::NumberTheory).   
- Visualization functions are provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
- Graphs and graph plots are provided by ["Graph"](https://raku.land/zef:antononcube/Graph).
- Data manipulation functions are provided by ["Data::Reshapers"](https://raku.land/zef:antononcube/Data::Reshapers).
- Data summarization functions are provided by ["Data::Summarizers"](https://raku.land/zef:antononcube/Data::Summarizers).
- Data translation functions (like `to-html`) are provided by ["Data::Translators"](https://raku.land/zef:antononcube/Data::Translators).

**Remark:** 
- Raku has built-in Number theory functions: `is-prime`, `mod`, `expmod`, `polymod`, `base`, `gcd`, `lcm`, etc. 
- The package "Math::NumberTheory" extends those functions and adds (many) more.
- "Math::NumberTheory" follows closely designs of Number theory functions in Wolfram Language (aka Mathematica.)

-----

## Setup

In [27]:
# Placed in in init.raku
#use Math::NumberTheory;
#use Math::NumberTheory::Utilities;

use JavaScript::D3::Utilities;

In [28]:
%% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [29]:
#%js
js-d3-list-line-plot(rand xx 40, background => 'none')

----

## Prime Pi

- `prime-pi` gives the number of primes $\pi(x)$ less than or equal to $x$.
- $\pi(x)$ is also known as *prime counting function*.
- $\pi(x)$ has the asymptotic expansion $x/log(x) + x/log^2(x)+2 x/log^3(x) + ... $ as $x \rightarrow \propto$.

In [ ]:
#% js
    my $limit = 200;
    my @data;
    @data .= append: (2..$limit).map({ %( group => 'x/log(x) + x/log(x)²', x => $_, y => $_ / log($_) +  $_ / log($_) ** 2 ) });
    @data .= append: (2..$limit).map({ %( group => 'x/log(x)', x => $_, y => $_ / log($_) ) });
    @data .= append: (2..$limit).map({ %( group => 'π(x)', x => $_, y => prime-pi($_) ) });
    js-d3-list-line-plot(@data, color-palette => 'Tableau10', background => 'none', width => 800, height => 300)

See [Prime number theorem](https://en.wikipedia.org/wiki/Prime_number_theorem) for details.

[Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) colored based on the difference in `prime-pi` values: 

In [31]:
my $method = 'Legendre';
my @mat = spiral-lattice(101).deepmap({ prime-pi($_ + 10, :$method) - prime-pi($_, :$method) });

deduce-type(@mat)

Vector(Vector(Atom((Int)), 101), 101)

Plot the spiral:

In [ ]:
#% js
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Rainbow', :!tooltip)

Generate a path based on the `prime-pi` sub:

In [ ]:
#% js
my @path = angle-path((1..1_000)».&prime-pi)».reverse.kv.map( -> $i, $p {[|$p, ($i div 100).Str]});
my %opts = color-scheme => 'Observable10', background => '#1F1F1F', :axes, :!legends;
js-d3-list-line-plot(@path, :800width, :500height, |%opts)

In [34]:
sink records-summary(@path, field-names => ["0", "1", "2"])

+-------------------------------+-------------------------------+----------------+
| 0                             | 1                             | 2              |
+-------------------------------+-------------------------------+----------------+
| Min    => -34.97751253833948  | Min    => -0.0667902539484635 | 4       => 100 |
| 1st-Qu => -20.12043698712268  | 1st-Qu => 4.077315500536086   | 6       => 100 |
| Mean   => -13.249046523603868 | Mean   => 8.370916136319657   | 1       => 100 |
| Median => -12.76276899793472  | Median => 7.517608575526668   | 7       => 100 |
| 3rd-Qu => -4.724627446571914  | 3rd-Qu => 12.51010823041242   | 0       => 100 |
| Max    => 8.73794847994785    | Max    => 18.68859172837683   | 3       => 100 |
|                               |                               | 8       => 100 |
|                               |                               | (Other) => 301 |
+-------------------------------+-------------------------------+----------------+


------

## Divisor sigma

The function `divisor-sigma(k,n)` gives the divisor function $\sigma_{k}(n)$.

- `divisor-sigma` is also known as the divisor function or sum‐of‐divisors function.
- `divisor-sigma(k,n)` is the sum of the $k^{th}$ powers of the divisors of $n$.
- For a number $n = u \times p_1^{e_1} ... p_{m}^{e_{m}}$ with a unit $u$ and $p_{i}$ primes, `divisor-sigma(k, n)` returns $(1 + p_1^{k} + p_1^{2 k} + ... + p_1^{e_1 k} )(1 + p_2^{k} + p_2^{2 k} + ... + p_1^{e_2 k})...$.

In [35]:
#%html
    my @cols = 1..3;
    (@cols X (^11)).map({ $_.tail => divisor-sigma($_.tail, $_.head) }).rotor(11)».Hash
    ==> { $_.kv.map(-> $k, %v { %v<n> = @cols[$k]; %v }) }()
    ==> to-html(field-names => ['n', |(^11)».Str])

n,0,1,2,3,4,5,6,7,8,9,10
1,1,1,1,1,1,1,1,1,1,1,1
2,2,3,5,9,17,33,65,129,257,513,1025
3,2,4,10,28,82,244,730,2188,6562,19684,59050


Here is a plot if $\sigma_i(n), i \in \{0, 1, 2\}$:

In [ ]:
#%js
    my @cols = "σ₀(n) Number of divisors", "σ₁(n) Sum of divisors", "σ₂(n) Sum of squares of divisors";
    my @data = ((^3) X (1..150)).map({ <group x y> Z=> [@cols[$_.head], $_.tail, divisor-sigma($_.tail, e => $_.head).log] })».Hash;
    js-d3-list-line-plot(@data,
        background => 'none',
        width => 1000,
        height => 300
    )

Recognize [amicable numbers](https://en.wikipedia.org/wiki/Amicable_numbers), two different numbers such that the sum of the proper divisors of each is equal to the other number:

In [37]:
my &is-amicable-pair = { $^a != $^b && divisor-sigma(1, $^a) - $^a == $^b && divisor-sigma(1, $^b) - $^b == $^a };

(1..300 X 1..300).grep({ &is-amicable-pair(|$_) })

((220 284) (284 220))

Much faster way to compute the amicable numbers:

In [38]:
my $n = 3_000;
my @sigmas = (1..$n).map({ divisor-sigma(1, $_) - $_ });
my &is-amicable-pair2 = { $^a != $^b && @sigmas[$^a-1] == $^b && @sigmas[$^b-1] == $^a  };

(1..$n).combinations(2).grep({ &is-amicable-pair2(|$_) })

((220 284) (1184 1210) (2620 2924))

In [39]:
factorial($n) / ( 2 * factorial($n - 2))

4498500

Plot a version of [Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) with the number divisors:

In [ ]:
#% js
my @mat = spiral-lattice(71).deepmap({ divisor-sigma(0, $_)});
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Plasma', :!tooltip)

Sunflower arrangement:

In [ ]:
#% js
my @sunflower = sunflower-embedding(5_000, with => { divisor-sigma(2, $_).log(2).round.Str }):d;
js-d3-list-plot(@sunflower, 
    background => 'none',
    point-size => 4,
    width => 900, height => 440, 
    :!axes, 
    :!legends,
    color-scheme => 'Tableau10',
    margins => {:20top, :20bottom, :250left, :250right}
 )

----

## Totient function 

The totient function $\varphi(n)$, also called Euler's totient function, is defined as the number of positive integers $≤ n$ that are relatively prime to (i.e., do not contain any factor in common with) $n$, where $1$ is counted as being relatively prime to all numbers. 

- `euler-phi` is also known as the [Euler totient function](https://mathworld.wolfram.com/TotientFunction.html) or _phi function_.
- Typically used in cryptography and in many applications in elementary number theory.
- `euler-phi(n)` counts positive integers up to `n` that are relatively prime to `n`.
- For the number $n = u \times p_1^{k_1} ... p_{m}^{m_1}$ with $u$ unit and $p_{i}$ primes, `euler-phi(n)` gives $n (1 - \frac{1}{p_1}) ... (1 - \frac{1}{p_{m}})$.

In [ ]:
#% js
js-d3-list-line-plot((^100)».&euler-phi, background => 'none', plot-label => "Euler's ϕ(x)", title-color => 'DimGray')

$\varphi(n)$ satisfies:

$$
\liminf_{n \rightarrow \propto} \varphi(n) \frac{\ln \ln n}{n} = e^{-\gamma} 
$$

See [MathWorld's "Totient Function"](https://mathworld.wolfram.com/TotientFunction.html).

In [ ]:
#% js
js-d3-list-plot(
    (2..5000).map({ euler-phi($_) * $_.log.log / $_ }), 
    background => 'none', 
    title => "Euler's ϕ(x)", 
    title-color => 'DimGray',
    point-size => 2
)

Plot of Ulam spiral where numbers are colored based on the values of `euler-phi`:

In [44]:
my $n = 47;
my @mats = 
    spiral-lattice($n, last-at => 'top-right').&flatten.hyper(:8degree).map({ euler-phi($_) }).rotor($n).cache,
    spiral-lattice($n, last-at => 'top-left').&flatten.hyper(:8degree).map({ euler-phi($_) }).rotor($n).cache,
;

@mats ==> deduce-type

Vector(Vector(Vector(Atom((Int)), 47), 47), 2)

In [ ]:
#% js
@mats.map({
    js-d3-matrix-plot($_, width => 400, height => 400, :!grid-lines, color-palette => 'Spectral', :!mesh, :!tooltip, :0margins)
}).join("\n")

Here we use Klauber triangle:

In [46]:
my $n = 97;
my @mat = triangle-matrix-embedding($n, missing-value => 0).&flatten.hyper(:8degree).map({ euler-phi($_) }).rotor(2 * $n - 1).cache;

@mat ==> deduce-type

Vector(Vector(Atom((Int)), 193), 97)

In [ ]:
#% js
js-d3-matrix-plot(@mat, width => 800, height => 400, :!grid-lines, color-palette => 'Inferno', :!mesh, :!tooltip, :0margins);

### Eight solutions

Find the only eight solutions of $\pi(x) = \varphi(x)$:

In [48]:
(1..100).grep({ prime-pi($_) == euler-phi($_) })

(2 3 4 8 10 14 20 90)

Possible intersections:

In [ ]:
#% js
my $n = 100;
js-d3-list-line-plot(
    [
        |(2..$n).map({ [$_, prime-pi($_), 'π'] }), 
        |(2..$n).map({ [$_, euler-phi($_), 'ϕ'] })
    ].flat(1), 
    background => 'none', 
    title => "Prime π(x) vs Euler's ϕ(x)", 
    title-color => 'DimGray')

---- 

## Bubble chart (again)

Sunflower embedding using bubble chart:

In [ ]:
#% js
my @sunflower = sunflower-embedding(2_000, with => { divisor-sigma(2, $_).log(2).round.Str }):d;

js-d3-bubble-chart(@sunflower.map({ my %h = $_.clone; %h<z> = (23 - %h<group>).log; %h }),
    z-range-min => 1, z-range-max => 8,
    background => 'none',
    width => 900, height => 410, 
    :!axes, 
    :!legends,
    :!tooltip,
    color-scheme => 'Pastel1',
    fill-color => 'DarkGray',
    margins => { :10top, :5bottom, :250left, :250right},
 )